In [1]:
import os
import psycopg
import json
import random
from datetime import timedelta
from dotenv import load_dotenv
import time

load_dotenv()  # take environment variables from .env.
if os.name == 'posix':
    tmpdir = '/tmp'
else:
    tmpdir = os.getenv('TMPDIR')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
# connect to database
conn = psycopg.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
cur = conn.cursor()


In [2]:
start_time = time.time()
    
sql_commands="""DROP TABLE IF EXISTS rides CASCADE;
DROP TABLE IF EXISTS cards CASCADE;
DROP TABLE IF EXISTS passengers CASCADE;
DROP TABLE IF EXISTS line_details CASCADE;
DROP TABLE IF EXISTS lines CASCADE;
DROP TABLE IF EXISTS exits CASCADE;
DROP TABLE IF EXISTS bus_stations CASCADE;
DROP TABLE IF EXISTS stations CASCADE;
DROP TABLE IF EXISTS bus_line_details CASCADE; 
DROP TABLE IF EXISTS bus_lines CASCADE;


CREATE TABLE IF NOT EXISTS cards(
        code VARCHAR(9) PRIMARY KEY CHECK (code ~ '^[0-9]{9}$'), 
        money MONEY NOT NULL, 
        create_time TIMESTAMP NOT NULL);
CREATE TABLE IF NOT EXISTS passengers (
        name VARCHAR(255) NOT NULL, 
        id_number VARCHAR(18) PRIMARY KEY NOT NULL CHECK (id_number ~ '^[0-9]{17}[0-9X]?$'), 
        phone_number VARCHAR(11) NOT NULL CHECK (phone_number ~ '^[0-9]{11}$'), 
        gender VARCHAR(255) NOT NULL, 
        district VARCHAR(255) NOT NULL);
CREATE TABLE IF NOT EXISTS lines (
        name VARCHAR(255)PRIMARY KEY, 
        start_time TIME, 
        end_time TIME, 
        mileage DOUBLE PRECISION, 
        color VARCHAR(255), 
        first_opening DATE, 
        url TEXT,
        intro TEXT);
CREATE TABLE IF NOT EXISTS stations (
        name VARCHAR(255) PRIMARY KEY, 
        district VARCHAR(255), 
        intro TEXT, 
        chinese_name VARCHAR(255));
CREATE TABLE IF NOT EXISTS rides (
        rail_user VARCHAR(255), 
        start_station VARCHAR(255), 
        end_station VARCHAR(255), 
        price DOUBLE PRECISION, 
        start_time TIMESTAMP, 
        end_time TIMESTAMP, 
        FOREIGN KEY (start_station) REFERENCES stations(name), 
        FOREIGN KEY (end_station) REFERENCES stations(name));
CREATE TABLE IF NOT EXISTS line_details (
        line_name VARCHAR(255), 
        station_name VARCHAR(255), 
        FOREIGN KEY (line_name) REFERENCES lines(name), 
        FOREIGN KEY (station_name) REFERENCES stations(name));
CREATE TABLE IF NOT EXISTS bus_stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255));
CREATE TABLE IF NOT EXISTS bus_lines (name VARCHAR(255) PRIMARY KEY);
CREATE TABLE IF NOT EXISTS bus_line_details (
        bus_line_name VARCHAR(255), 
        bus_station_name VARCHAR(255), 
        FOREIGN KEY (bus_station_name) REFERENCES bus_stations(name));
CREATE TABLE IF NOT EXISTS exits (
        station_name VARCHAR(255), 
        name VARCHAR(255),
        textt TEXT,
        FOREIGN KEY (station_name) REFERENCES stations(name),
        PRIMARY KEY (station_name, name));"""+f"""
COPY cards FROM '/{tmpdir}/cards.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
COPY passengers FROM '/{tmpdir}/passengers.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
COPY lines FROM '/{tmpdir}/lines.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
COPY stations FROM '/{tmpdir}/stations.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
COPY rides FROM '/{tmpdir}/rides.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
COPY line_details FROM '/{tmpdir}/line_details.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
COPY exits FROM '/{tmpdir}/exits.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
CREATE TEMP TABLE tmp_bus_stations AS SELECT * FROM bus_stations LIMIT 0;
        COPY tmp_bus_stations FROM '/{tmpdir}/bus_stations.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
        INSERT INTO bus_stations SELECT * FROM tmp_bus_stations ON CONFLICT DO NOTHING;
CREATE TEMP TABLE tmp_bus_line_details AS SELECT * FROM bus_line_details LIMIT 0;
        COPY tmp_bus_line_details FROM '/{tmpdir}/bus_line_details.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
        INSERT INTO bus_line_details (SELECT DISTINCT * FROM tmp_bus_line_details) ON CONFLICT DO NOTHING;
CREATE TEMP TABLE tmp_exit_details AS SELECT * FROM exit_details LIMIT 0;
        COPY tmp_exit_details FROM '/{tmpdir}/exit_details.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
        INSERT INTO exits (SELECT station_name, name, bus_station_name AS textt FROM tmp_exit_details) ON CONFLICT DO NOTHING;
        INSERT INTO exit_details (SELECT * FROM tmp_exit_details) ON CONFLICT DO NOTHING;
"""
# Execute the SQL commands
cur.execute(sql_commands)
# Record the end time
end_time = time.time()
# Commit the changes
conn.commit()
execution_time = end_time - start_time

print(f"The SQL commands took {execution_time} seconds to execute.")

The SQL commands took 0.831416130065918 seconds to execute.
